In [28]:
import zlib
import pickle

def check_pkz_integrity(file_path):
    try:
        with open(file_path, 'rb') as f:
            compressed_data = f.read()
            uncompressed_data = zlib.decompress(compressed_data)
            # Tente desserializar os dados
            _ = pickle.loads(uncompressed_data)
        print("O arquivo PKZ está íntegro.")
    except Exception as e:
        print(f"Erro ao verificar integridade do arquivo PKZ: {str(e)}")

# Substitua 'caminho/para/seu/arquivo.pkz' pelo caminho real para o seu arquivo PKZ.
check_pkz_integrity('/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/spin21.pkz')


Erro ao verificar integridade do arquivo PKZ: invalid load key, '\x00'.


In [26]:


import os
import pickle
import zlib
import numpy as np
from netCDF4 import Dataset, date2num, num2date
import datetime
import joblib
import cftime

file_path = '/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/spin21.pkz'
pkz_folder = '/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/'
output_folder = '/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/output_folder'

with open(file_path, 'rb') as fh:
   dt = pickle.load(fh)
   
   print(dt.keys()) # list the available keys for the ouputs
   # 'calendar', 'time_unit', 'sind', 'eind'
   print(dt['calendar'])
   print(dt['time_unit'])
   print(dt['sind'])
   print(dt['eind'])

def load_pkz(file_path):
    with open(file_path, 'rb') as f:
        compressed_data = f.read()
    data = pickle.loads(zlib.decompress(compressed_data))
    return data

def create_lband(res=0.5):
    #creates an array of longitudes based on a spacing of res. that is 0.5º
    lon = np.arange(-179.75, 180, res, dtype=np.float64)[201:272]
    lat = np.arange(89.75, -90, -res, dtype=np.float64)[160:221][::-1]

    #defines the lower and upper bounds of each latitude band. 
    half = res / 2.0

    #the following vars refer to the latitude and longitude 
    # intervals associated with each specific point in a geographic grid
    latbnd = np.array([[l - half, l + half] for l in lat])
    lonbnd = np.array([[l - half, l + half] for l in lon])
    
    return lat, latbnd, lon, lonbnd

def set_historical_stime(new_descr=True):

    global TIME_UNITS, CALENDAR
    TIME_UNITS = "days since 1860-01-01 00:00:00"
    CALENDAR = "proleptic_gregorian"

def get_var_metadata(var):

    vunits = {'header': ['long_name', 'unit', 'standart_name'],

              'tsoil': ['soil_temperature', 'celcius', 'soil_temp'],
              'emaxm': ['potent. evapotrasnpiration', 'kg m-2 day-1', 'etpot'],
              'aresp': ['autothrophic respiration', 'kg m-2 year-1', 'ar'],
              'photo': ['gross primary productivity', 'kg m-2 year-1', 'gpp'],
              'npp': ['net primary productivity', 'kg m-2 year-1', 'npp'],
              'cawood': ['C in woody tissues', 'kg m-2', 'cawood'],
              'cfroot': ['C in fine roots', 'kg m-2', 'cfroot'],
              'cleaf': ['C in leaves', 'kg m-2', 'cleaf']}

    out = {}
    for v in var:
        out[v] = vunits[v]
    return out

# Função para converter dados de PKZ para NetCDF
def convert_pkz_to_netcdf(pkz_folder, output_folder):
    
    
    # Certifique-se de que a pasta de saída exista
    os.makedirs(output_folder, exist_ok=True)

    set_historical_stime()

    NO_DATA = [-9999.0]
    time_units = "days since 1860-01-01 00:00:00"
    calendar = "proleptic_gregorian"
   
    #prepare lat/lon
    geo_v = create_lband()
    lat = geo_v[0]
    lat_bnds = geo_v[1]
    lon = geo_v[2]
    lon_bnds = geo_v[3]

    # Variáveis (from pkz file)
    variables = ['emaxm', 'tsoil', 'photo', 'aresp', 'npp', 'cleaf', 'cawood', 'cfroot']

    # Resolução da grade
    resolucao = 0.5

    # Calcula o número total de pontos de grade
    num_latitudes = len(lat)
    num_longitudes = len(lon)

    # Certifique-se de que a pasta de saída exista
    os.makedirs(output_folder, exist_ok=True)
    
    var_attrs = get_var_metadata(variables)

    for i, v in enumerate(variables):
        
        nc_filename = os.path.join(output_folder, f"{v}.nc")
        with Dataset(nc_filename, 'w', format='NETCDF4') as nc:

    #         # Dimensões e variáveis
            nc.createDimension('time', None)
            nc.createDimension('lat', num_latitudes)
            nc.createDimension('lon', num_longitudes)

            time_var = nc.createVariable('time', np.float64, ('time',))
            lat_var = nc.createVariable('lat', lat.dtype, ('lat',))
            lon_var = nc.createVariable('lon', lon.dtype, ('lon',))
    #           # Escolhe um nome para a variável NetCDF
            nc_var_name = f"{v}_data"
            
            data_var = nc.createVariable(nc_var_name, datatype=np.float32, dimensions = ('time', 'lat', 'lon'),
                                         zlib=True, fill_value=NO_DATA[0], fletcher32=True) 

    #         # Adiciona atributos
            time_var.units = time_units
            time_var.calendar = calendar
            lat_var.units = 'degrees_north'
            lon_var.units = 'degrees_east'
            
            data_var.long_name = var_attrs[v][0]
            data_var.units     = var_attrs[v][1]
            data_var.standard_name = var_attrs[v][2]
            data_var.missing_value = NO_DATA

            # # Loop através dos arquivos PKZ
            # for root, dirs, files in os.walk(pkz_folder):
            #    for file in files:
            #         if file.endswith('.pkz'):
            #             file_path = os.path.join(root, file)

                        
            gridcell_data = load_pkz('/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/spin01.pkz')


convert_pkz_to_netcdf(pkz_folder, output_folder)

UnpicklingError: invalid load key, 'x'.

In [9]:
import os
import pickle
import zlib
import numpy as np
from netCDF4 import Dataset, date2num, num2date
import datetime

# Função para carregar dados de um arquivo PKZ
def load_pkz(file_path):
    with open(file_path, 'rb') as f:
        compressed_data = f.read()
    data = pickle.loads(zlib.decompress(compressed_data))
    return data

# Função para converter dados de PKZ para NetCDF
def convert_pkz_to_netcdf(pkz_folder, output_folder):
    # Certifique-se de que a pasta de saída exista
    os.makedirs(output_folder, exist_ok=True)

    # Variáveis
    variables = ['photo', 'aresp', 'npp', 'lai', 'wue', 'cue', 'vcmax', 'sla', 'nupt', 'pupt', 'ls']

    # Loop através das variáveis
    for variable in variables:
        # Inicializa o NetCDF
        nc_filename = os.path.join(output_folder, f"{variable}.nc")
        with Dataset(nc_filename, 'w', format='NETCDF4') as nc:
            # Dimensões
            nc.createDimension('time', None)  # Pode ser alterado para o número total de dias
            nc.createDimension('lat', 61)     # Altere para a resolução real da grade
            nc.createDimension('lon', 71)     # Altere para a resolução real da grade

            # Variáveis
            time_var = nc.createVariable('time', 'f8', ('time',))
            lat_var = nc.createVariable('lat', 'f4', ('lat',))
            lon_var = nc.createVariable('lon', 'f4', ('lon',))
            data_var = nc.createVariable(variable, 'f4', ('time', 'lat', 'lon'), fill_value=-9999.0)

            # Adiciona atributos
            time_var.units = 'days since 1900-01-01'
            time_var.calendar = 'gregorian'
            lat_var.units = 'degrees_north'
            lon_var.units = 'degrees_east'
            data_var.units = 'your_unit'  # Substitua pelo valor real

            # Loop através dos arquivos PKZ
            for root, dirs, files in os.walk(pkz_folder):
                for file in files:
                    if file.endswith('.pkz'):
                        file_path = os.path.join(root, file)

                        
                        gridcell_data = load_pkz(file_path)

                        # Extrai as coordenadas da célula de grade do nome do arquivo ou dos metadados
                        # Substitua isso com base na sua estrutura de dados
                        lat, lon = extract_lat_lon_from_filename(file)

                        # Converte as datas para números do tipo float (dias desde 1900-01-01)
                        dates = [date2num(datetime.datetime.strptime(date_str, '%Y-%m-%d'), 'days since 1900-01-01') for date_str in gridcell_data['dates']]

                        # Preenche os dados NetCDF
                        time_var[:] = dates
                        lat_var[:] = lat
                        lon_var[:] = lon
                        data_var[:, lat, lon] = gridcell_data[variable]

                        print(f"Variable {variable} processed for lat={lat}, lon={lon}")

# Função de exemplo para extrair lat e lon do nome do arquivo (ajuste conforme necessário)
def extract_lat_lon_from_filename(file):
    # Supondo que o nome do arquivo seja algo como "gridcell_LAT_LON.pkz"
    parts = file.split('_')
    lat = float(parts[1])
    lon = float(parts[2].split('.')[0])
    return lat, lon


convert_pkz_to_netcdf(pkz_folder, output_folder)


UnpicklingError: invalid load key, '\x00'.